In [8]:
import logging
import numpy as np
from typing import Tuple, Dict, Annotated
from tensorflow.keras.models import Model

In [10]:
class ModelEvaluate:
    """
    Class for model evaluation and calculating predictions
    """
    def __init__(self, 
                 model: Model, 
                 test_dataset: Dict[str, Dict[str, np.ndarray]]
                 ) -> None:
        """
        Args:
            model: Trained tensorflow.keras model
            test_dataset: Dictionary test dataset
        """
        self.model = model
        self.test_targets = test_dataset["targets"]
        self.test_features = test_dataset["input_features"]


    def model_predict(self) -> Tuple[np.ndarray, np.ndarray]:
        """
        Predict targets on a test dataset
        
        Returns:
            Tuple[np.ndarray, np.ndarray]:
                - 'home_score' predictions
                - 'away_score' predictions
        """
        try:
            logging.info("Data prediction:")
            predictions = self.model.predict(self.test_features)
            home_score_predictions, away_score_predictions = predictions
            logging.info("Successfully predicted data on the model")
            return home_score_predictions, away_score_predictions
        except Exception as e:
            logging.error(f"Error in model prediction: {e}")
            raise e
        
    def calculate_metrics(self, metrics: Dict[str, str]) -> Tuple[
        Annotated[float, "loss"],
        Annotated[float, "home_loss"],
        Annotated[float, "away_loss"],
        Annotated[float, "home_evaluation"],
        Annotated[float, "away_evaluation"]
    ]:
        """
        Evaluates the model

        Args:
            metrics: Dictionary with names of loss and metrics algorithms used in model
        Returns:
            Tuple[
        Annotated[float, "loss"],
        Annotated[float, "home_loss"],
        Annotated[float, "away_loss"],
        Annotated[float, "home_evaluation"],
        Annotated[float, "away_evaluation"]
            ]:
            - loss: loss value
            - home_loss: 'home_score' loss value
            - away_loss: 'away_score' loss value
            - home_evaluation: 'home_score' evaluation metric
            - away_evaluation: 'away_score' evaluation metric
        """
        try:
            logging.info("\nCalculating metrics:")
            loss, home_loss, away_loss, home_evaluation, away_evaluation = self.model.evaluate(self.test_features, self.test_targets)
            logging.info(f"\nLoss ({metrics['loss']}): {loss} \n"
                         f"'home_score' loss ({metrics['loss']}): {home_loss} \n"
                         f"'away_score' loss ({metrics['loss']}): {away_loss} \n"
                         f"'home_score' {metrics['metrics']}: {home_evaluation} \n"
                         f"'away_score' {metrics['metrics']}: {away_evaluation}")
        except Exception as e:
            logging.error(f"Error in model evaluation: {e}")
            raise e 
        
    def round_results(self, predictions: np.ndarray) -> np.ndarray:
        """
        Changes continuous into discret values
        
        Args:
            predictions: Predictions with continuous values
        Returns:
            np.ndarray: Predictions with discret values
        """
        try:
            predictions_rounded = np.round(predictions)
            predictions_reshaped = predictions_rounded.reshape((len(predictions_rounded)))
            return predictions_reshaped
        except Exception as e:
            logging.error(f"Error while converting continuous into discret values: {e}")
            raise e
    
    def select_one_category(self, predictions: np.ndarray) -> np.ndarray:
        """
        Selects the category with the highest probability for each row of predictions
        
        Args:
            predictions: A 2D array, where each row contains the predicted probabilities for each class
        Returns:
            np.ndarray: An array of predicted class indicies, where each element represents 
                        the index of class with the highest probability for the correcponding row
        """
        try:
            predictions_transformed = np.argmax(predictions, axis=1)
            return predictions_transformed
        except Exception as e:
            logging.error(f"Error while celecting the category with the highest probability: {e}")
            raise e